In [17]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
from networkx.algorithms import community

 Influence Diffusion (Independent Cascade Model)

In [ ]:
def simulate_ic(G, seeds, p=0.1):
    active = set(seeds)
    newly_active = set(seeds)

    while newly_active:
        next_new = set()
        for u in newly_active:
            for v in G.neighbors(u):
                if v not in active:
                    if random.random() < p:
                        next_new.add(v)
        newly_active = next_new
        active |= newly_active
    return active

Influence Diffusion (Linear Threshold Model)

In [ ]:
def simulate_lt(G, seeds):
    thresholds = {node: random.uniform(0, 1) for node in G.nodes()}
    active = set(seeds)
    changed = True

    while changed:
        changed = False
        for node in G.nodes():
            if node not in active:
                neighbors = list(G.predecessors(node)) if G.is_directed() else list(G.neighbors(node))
                active_neighbors = len([n for n in neighbors if n in active])
                if neighbors:
                    if (active_neighbors / len(neighbors)) >= thresholds[node]:
                        active.add(node)
                        changed = True
    return active

Main analysis function

In [ ]:
def analyze_influence(graphs, ks=[5,10,20,25,30], trials=10, p=0.1, model="IC"):
    results = {}
    tables = {}
    for name, G in graphs.items():
        n = G.number_of_nodes()
        cent = {
            'Degree': dict(nx.degree_centrality(G)),
            'Closeness': dict(nx.closeness_centrality(G)),
            'Betweenness': dict(nx.betweenness_centrality(G)),
            'PageRank': dict(nx.pagerank(G))
        }
        results[name] = {}
        data = []
        for measure, scores in cent.items():
            sorted_nodes = sorted(scores, key=lambda x: scores[x], reverse=True)
            avg_fracs = []
            for k in ks:
                seeds = sorted_nodes[:k]
                influences = []
                for _ in range(trials):
                    if model == "IC":
                        active = simulate_ic(G, seeds, p)
                    elif model == "LT":
                        active = simulate_lt(G, seeds)
                    influences.append(len(active))
                avg_inf = np.mean(influences)
                avg_fracs.append(avg_inf / n)
                data.append([measure, k, avg_inf / n])
            results[name][measure] = avg_fracs
        tables[name] = pd.DataFrame(data, columns=['Centrality', 'k', 'Influence'])
    return results, tables

Plotting function

In [ ]:
def plot_results(results, ks, model="IC"):
    for graph_name, measures in results.items():
        plt.figure(figsize=(8, 5))
        for measure, fracs in measures.items():
            plt.plot(ks, fracs, marker='o', label=measure)
        plt.title(f'{model} Model Influence on {graph_name}')
        plt.xlabel('Seed Set Size (k)')
        plt.ylabel('Average Influence (fraction)')
        plt.xticks(ks)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

In [ ]:
def plot_comparison_bar(tables, model):
    for name, df in tables.items():
        pivot_df = df.pivot(index='k', columns='Centrality', values='Influence')
        pivot_df.plot(kind='bar', figsize=(10, 6))
        plt.title(f'Influence Comparison (Bar) - {model} Model - {name}')
        plt.xlabel('Seed Set Size (k)')
        plt.ylabel('Average Influence (fraction)')
        plt.legend(title='Centrality')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

In [ ]:
def plot_centrality_comparison(results_ic, results_lt, ks):
    for graph_name in results_ic:
        plt.figure(figsize=(10, 6))
        for centrality in results_ic[graph_name]:
            plt.plot(ks, results_ic[graph_name][centrality], label=f'IC - {centrality}', linestyle='--')
            plt.plot(ks, results_lt[graph_name][centrality], label=f'LT - {centrality}', linestyle='-')
        plt.title(f'IC vs LT Influence Comparison - {graph_name}')
        plt.xlabel('Seed Set Size (k)')
        plt.ylabel('Average Influence (fraction)')
        plt.xticks(ks)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()